In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_price(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  price = soup.find('span',{'itemprop': 'price'})['content']
  return price


In [ ]:
print(get_price('https://www.ebay.com/itm/393514670181?_trkparms=amclksrc%3DITM%26aid%3D1110010%26algo%3DHOMESPLICE.DISCCARDS%26ao%3D1%26asc%3D20200818141627%26meid%3D35d3db7929d24681ae460bcffcef9ec3%26pid%3D101110%26rk%3D1%26rkt%3D1%26sd%3D234145481103%26itm%3D393514670181%26pmt%3D1%26noa%3D0%26pg%3D2563228%26algv%3Ddefault%26brand%3DPUMA&_trksid=p2563228.c101110.m1982&amdata=cksum%3A39351467018135d3db7929d24681ae460bcffcef9ec3%7Cenc%3AAQAGAAAB4LAfpiaX1XNFiDFtpTEdZWAf3k%252BG0cxQdDyso%252BsfnJRewPxG54S3DsmowvuLtaYAL074LDQn8f4BO%252FQ4tnhMeCvyvItOMvn9aGVeR0jIgBiCJKa5jp6Qg1tJFMDfSONIILjnzUX%252FKOKhppxys0f9I%252BhBJ7txJuN4sF4CAUDbMdW7mKbQbYhpUMJi5Ezm%252FRh7WDuJ%252FuD%252FWp4JgcihztnZkJJdE3CEvi5CylVvWmrSLJX30dPOgNFdI%252BBfMJY4vegu%252FqE%252FUIaUVAg8GtZxGtDvMSj59qitBkZt%252F2Ca%252BOr8%252FHCTDBnDMklT7uinO3rowg3kguth%252FHYBMbLNHLR4zlcxS01QbVicZjnG7WlsxuvEcwwVuO0r3zPQ0oYhrjy0P1PuKktzVwA2ZySbCaTEolkwcuJZhNRyJNwgoTAAcn3c8t9FFMoPrjKC%252B9sEi5xsQbxCP33HknaJY1JzR05ZbcHyRWDyfj7YQViK9%252BN%252F4wAQDqkzxpncA6zwjCBmRefCVqs5%252BseobqNDkB3NM358%252BasmsJShdxMnLtDFo1%252BIRRPztK2GZTDxSlBS7PjPZAS2lZH0CJE4mBgBqa2VhF1hY%252Fm%252FEjh8uwEqY%252FTZD484aCkIXKN%252F7ztdjmv4vPtI1n4MPc42sw%253D%253D%7Campid%3APL_CLK%7Cclp%3A2563228'))

82.99


### Using selenium


In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('useAutomationExtention', False)
chrome_options.add_argument('--ifnore-certificate-errors')
chrome_options.add_argument('--start-maximized')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 954 kB 5.2 MB/s 
     |████████████████████████████████| 356 kB 36.8 MB/s 
     |████████████████████████████████| 138 kB 56.2 MB/s 
     |████████████████████████████████| 3.5 MB 29.4 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [2]:
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd 
import os
import time

In [3]:
#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
#FireFoxDriverPath = os.path.join(os.getcwd(), 'Drivers', 'geckodriver.exe')
#FireFoxProfile = webdriver.FirefoxProfile()
#FireFoxProfile.set_preference("general.useragent.override", user_agent)
url = "https://www.ebay.com/"
browser = wd
browser.implicitly_wait(7)
browser.get(url)

search_keyword = 'Playstation 5'
search_field = browser.find_element_by_id('gh-ac')
search_field.clear()
search_field.send_keys(search_keyword)
search_field.send_keys(Keys.ENTER)

total_result = int(browser.find_element_by_class_name('srp-controls__count-heading').find_element_by_class_name('BOLD').text.replace(',', ''))
ebay_listings = []

current_page = 1
to_continue = True

In [17]:
while to_continue:
    try:
        print('Processing page {0}'.format(current_page))

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        item_list = soup.find_all('li', 's-item--watch-at-corner')
        
        for listing in item_list:
            product_detail = {}

            product_detail['product title'] = listing.h3.text
            product_detail['product url'] = listing.a['href']
            
            listing_subtitles = listing.find_all('div', 's-item__subtitle')
            if len(listing_subtitles) == 2:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = listing_subtitles[1].text
            elif len(listing_subtitles) == 1:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = None
            else:
                listing_subtitle_1 = None
                listing_subtitle_2 = None

            product_detail['subtitle1'] = listing_subtitle_1
            product_detail['subtitle2'] = listing_subtitle_2


            # Star rating and product review count
            try:
                x_star_rating = listing.find('div', 'x-star-rating')
                stars = len(x_star_rating.find_all('svg', 'icon--star-full-small'))
                if int(stars) > 4: print(stars) 
                review_count = int(listing.find('span', 's-item__reviews-count').span.text.replace( ' product ratings', ''))
                
                product_detail['stars'] = stars
                product_detail['product review_count'] = review_count
            except Exception as e:
                product_detail['stars'] = ''
                product_detail['product review_count'] = ''

            product_detail['price'] = listing.find('span', 's-item__price').text

            try:
                product_detail['shipping detail'] = listing.find('span', 's-item__logisticsCost').text
            except AttributeError:
                product_detail['shipping detail'] = 'Not Available'
                
            ebay_listings.append(product_detail)


        time.sleep(1)
        browser.find_element_by_xpath('//a[@class="pagination__item" and text()="{0}"]'.format(current_page+1)).click()
        current_page +=1

        if current_page > 5:
            raise Exception('Stop')

    except NoSuchElementException:
        print('Last page {0}'.format(current_page))
        to_continue = False



In [18]:
df = pd.DataFrame(ebay_listings)
df.head()

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail
0,Sony PlayStation 5 PS5 Console Disc Version *N...,https://www.ebay.com/itm/333896636458?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,049.00",Not Available
1,PS5 Sony PlayStation 5 Console Disc Version,https://www.ebay.com/itm/324483721021?epid=190...,Pre-Owned · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$859.99,Free shipping
2,Sony PlayStation 5 Console Disc Version (PS5) ...,https://www.ebay.com/itm/144223680923?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$949.99,Free shipping
3,Sony PlayStation 5 PS5 Console Disc Version Bl...,https://www.ebay.com/itm/193955663603?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$899.99,Free shipping
4,New ListingSony PlayStation 5 PS5 DISC Edition...,https://www.ebay.com/itm/203647656449?hash=ite...,Brand New · Sony PlayStation 5,None,,,$760.00,+$23.77 shipping


In [19]:
df['amount'] = df.price.apply(lambda x: int("".join(filter(str.isdigit, x))))

In [20]:
df.sort_values(by='amount')

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail,amount
7,Turtle Beach Elite Atlas Pro Performance Heads...,https://www.ebay.com/itm/324391510282?hash=ite...,None,None,,,$59.95,Free shipping,5995
8,"Razer Kraken Gaming Headset For PC, PS4, PS5, ...",https://www.ebay.com/itm/144123396447?hash=ite...,None,None,,,$59.99,Free shipping,5999
9,SteelSeries Arctis 5 RGB Wired Gaming Headset ...,https://www.ebay.com/itm/334129396663?hash=ite...,None,None,,,$89.95,Free shipping,8995
191,New ListingSony PlayStation 5 PS5 Blu-Ray Edit...,https://www.ebay.com/itm/324827522681?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$102.50,+$77.61 shipping,10250
113,New Listing🎁Sony PlayStation 5 Console🎁Digital...,https://www.ebay.com/itm/265357244952?epid=250...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,461,$202.50,+$50.00 shipping,20250
...,...,...,...,...,...,...,...,...,...
129,New Listingplaystation 5 console,https://www.ebay.com/itm/304180922459?_trkparm...,Brand New · Sony PlayStation 5,None,,,"$1,124.00",+$10.00 shipping,112400
137,Sony PlayStation 5 Console PS5 [Disc Version] ...,https://www.ebay.com/itm/284478117917?epid=190...,Brand New · Sony PlayStation 5,None,0,1976,"$1,151.00",+$30.15 shipping,115100
64,Sony PlayStation 5 PS5 Console Disc Version BR...,https://www.ebay.com/itm/203581141972?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.95",Free shipping,119995
103,PS5 Sony Playstation 5 Console Disc Version 🔥B...,https://www.ebay.com/itm/304128005531?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.99",Free shipping,119999


In [11]:
#df.to_excel('playstation 5 listings.xlsx', index=False)
print(df.iloc[103])

product title           PS5 Sony Playstation 5 Console Disc Version 🔥B...
product url             https://www.ebay.com/itm/304128005531?epid=190...
subtitle1               Brand New · Sony PlayStation 5 · Sony PlayStat...
subtitle2                                                            None
stars                                                                   0
product review_count                                                 1976
price                                                           $1,199.99
shipping detail                                             Free shipping
amount                                                             119999
Name: 103, dtype: object


In [ ]:
# https://towardsdatascience.com/scraping-multiple-amazon-stores-with-python-5eab811453a8

# scrapping Amazon



In [3]:
def get_url(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [4]:
url = get_url('jetson nano')
print(url)

https://www.amazon.com.mx/s?k=jetson+nano&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1


In [5]:
wd.get(url)

In [6]:
# Extract the collection
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [7]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [8]:
len(results)

60

In [9]:
# prototype the record
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url = 'https://www.amazon.com.mx' + atag.get('href')

In [11]:
# price
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [14]:
# stars
rating = item.i.text

In [19]:
# find the reviews
review_count = item.find('span', {'class':'a-size-base'}).text

In [20]:
# Generalize pattern

def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  # price
  price_parent = item.find('span', 'a-price')
  price = price_parent.find('span', 'a-offscreen').text

  # reviews and rating
  rating = item.i.text
  review_count = item.find('span', {'class':'a-size-base'}).text

  result = (description, price, rating, review_count, url)

  return result
  

In [21]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
  records.append(extract_record(item))

AttributeError: ignored